# PDF text extraction playground

In [1]:
import copy
import re
import glob
import pprint

import pdfplumber
import nltk

In [2]:
DELIM = '_'

In [35]:
# PDF Plumber

files = glob.glob('files/pdf/*')

for file in files:
    with pdfplumber.open(file) as pdf:
        
        # extracting all text
        text = ''
        for page in pdf.pages:
            try:
                text += page.extract_text()
            except TypeError:
                continue

        # extracting name
        name = re.findall(r'A\/RES\/\d{2}\/\d{1,3}', text)
        name = list(set(name))
        
        if len(name) == 0:
            continue
        
        if len(name) > 1:
            print('[ACTION]: multiple res symbols:', name, 'choose index; file', file)
            index = int(input('>>>'))
            name = list(name[index])

        # in case a name isn't found or multiple are found
        # if len(name) != 1:
        #     print('[INFO]: multiple / 0 resolutions found:', name, 'file:', file)
        #     print('[ACTION]: choose index or \'c\' to custom-name')
        #     inpt = input('>>> ')
    
        #     if inpt == 'c':
        #         print('[ACTION]: name (no extension)')
        #         name = input('>>> ')

        #     try:
        #         name = name[int(inpt)]
        #     except ValueError:
        #         continue
        # else:
        #     name = name[0]
        
        name = name[0].replace('/', DELIM)

        # save file with text
        with open('files/text/' + name + '.txt', 'w') as txt:
            txt.write(text)
        
        print('[INFO]: saved file: ', name)

[INFO]: saved file:  A_RES_58_283
[INFO]: saved file:  A_RES_65_276
[INFO]: saved file:  A_RES_50_199
[INFO]: saved file:  A_RES_50_45
[INFO]: saved file:  A_RES_50_47
[INFO]: saved file:  A_RES_65_275
[INFO]: saved file:  A_RES_51_171
[INFO]: saved file:  A_RES_51_9
[INFO]: saved file:  A_RES_61_245
[INFO]: saved file:  A_RES_67_140
[INFO]: saved file:  A_RES_57_37
[INFO]: saved file:  A_RES_50_195
[INFO]: saved file:  A_RES_49_234
[INFO]: saved file:  A_RES_65_313
[INFO]: saved file:  A_RES_71_221
[INFO]: saved file:  A_RES_72_279
[INFO]: saved file:  A_RES_66_261
[INFO]: saved file:  A_RES_65_314
[INFO]: saved file:  A_RES_56_13
[INFO]: saved file:  A_RES_51_95
[INFO]: saved file:  A_RES_73_344
[INFO]: saved file:  A_RES_51_198
[INFO]: saved file:  A_RES_59_115
[INFO]: saved file:  A_RES_58_101
[INFO]: saved file:  A_RES_48_53
[INFO]: saved file:  A_RES_74_214
[INFO]: saved file:  A_RES_56_60
[INFO]: saved file:  A_RES_71_205
[INFO]: saved file:  A_RES_58_239
[INFO]: saved file:  A_

>>> 1


[INFO]: saved file:  A
[INFO]: saved file:  A_RES_65_49
[INFO]: saved file:  A_RES_48_133
[INFO]: saved file:  A_RES_53_212
[INFO]: saved file:  A_RES_70_247
[INFO]: saved file:  A_RES_56_260
[INFO]: saved file:  A_RES_54_70
[INFO]: saved file:  A_RES_70_1
[INFO]: saved file:  A_RES_55_280
[INFO]: saved file:  A_RES_67_261
[INFO]: saved file:  A_RES_60_22
[INFO]: saved file:  A_RES_57_304
[INFO]: saved file:  A_RES_48_171
[INFO]: saved file:  A_RES_69_134
[INFO]: saved file:  A_RES_65_279
[INFO]: saved file:  A_RES_71_115
[INFO]: saved file:  A_RES_68_112
[INFO]: saved file:  A_RES_64_169
[INFO]: saved file:  A_RES_52_226
[INFO]: saved file:  A_RES_48_189
[INFO]: saved file:  A_RES_69_274
[INFO]: saved file:  A_RES_68_265
[INFO]: saved file:  A_RES_54_23
[INFO]: saved file:  A_RES_68_218
[INFO]: saved file:  A_RES_52_101
[INFO]: saved file:  A_RES_56_296
[INFO]: saved file:  A_RES_49_158
[INFO]: saved file:  A_RES_71_280
[INFO]: saved file:  A_RES_63_199
[INFO]: saved file:  A_RES_49_1

>>> 0


[INFO]: saved file:  A
[INFO]: saved file:  A_RES_74_307
[INFO]: saved file:  A_RES_73_340
[INFO]: saved file:  A_RES_51_239
[INFO]: saved file:  A_RES_57_258
[INFO]: saved file:  A_RES_49_54
[INFO]: saved file:  A_RES_74_156
[INFO]: saved file:  A_RES_65_2
[INFO]: saved file:  A_RES_50_3
[INFO]: saved file:  A_RES_66_249
[INFO]: saved file:  A_RES_54_277
[INFO]: saved file:  A_RES_52_174
[INFO]: saved file:  A_RES_57_20
[INFO]: saved file:  A_RES_53_209
[INFO]: saved file:  A_RES_54_79
[INFO]: saved file:  A_RES_56_132
[INFO]: saved file:  A_RES_68_223
[INFO]: saved file:  A_RES_57_19
[INFO]: saved file:  A_RES_73_301
[INFO]: saved file:  A_RES_48_244
[INFO]: saved file:  A_RES_59_271
[INFO]: saved file:  A_RES_55_42
[INFO]: saved file:  A_RES_58_57
[INFO]: saved file:  A_RES_54_282
[INFO]: saved file:  A_RES_59_165
[INFO]: saved file:  A_RES_48_63
[INFO]: saved file:  A_RES_48_87
[INFO]: saved file:  A_RES_61_225
[INFO]: saved file:  A_RES_65_255
[INFO]: saved file:  A_RES_61_252
[IN

...

[INFO]: saved file:  A
[INFO]: saved file:  A_RES_71_217
[INFO]: saved file:  A_RES_63_192
[INFO]: saved file:  A_RES_49_190
[INFO]: saved file:  A_RES_57_173
[INFO]: saved file:  A_RES_60_20
[INFO]: saved file:  A_RES_65_136
[INFO]: saved file:  A_RES_49_147
[INFO]: saved file:  A_RES_57_25
[INFO]: saved file:  A_RES_65_135
[INFO]: saved file:  A_RES_74_187
[INFO]: saved file:  A_RES_60_246
[INFO]: saved file:  A_RES_71_288
[INFO]: saved file:  A_RES_71_138
[INFO]: saved file:  A_RES_56_501
[INFO]: saved file:  A_RES_48_34
[INFO]: saved file:  A_RES_59_26
[INFO]: saved file:  A_RES_52_26
[INFO]: saved file:  A_RES_53_178
[INFO]: saved file:  A_RES_71_140
[INFO]: saved file:  A_RES_50_124
[INFO]: saved file:  A_RES_60_173
[INFO]: saved file:  A_RES_56_6
[INFO]: saved file:  A_RES_72_129
[INFO]: saved file:  A_RES_56_8
[INFO]: saved file:  A_RES_63_307
[INFO]: saved file:  A_RES_52_142
[INFO]: saved file:  A_RES_52_29
[INFO]: saved file:  A_RES_54_6
[INFO]: saved file:  A_RES_48_180
[IN

>>> 0


[INFO]: saved file:  A
[INFO]: saved file:  A_RES_64_213
[INFO]: saved file:  A_RES_63_39
[INFO]: saved file:  A_RES_74_269
[INFO]: saved file:  A_RES_48_123
[INFO]: saved file:  A_RES_52_202
[INFO]: saved file:  A_RES_64_211
[INFO]: saved file:  A_RES_53_145
[INFO]: saved file:  A_RES_48_177
[INFO]: saved file:  A_RES_74_2
[INFO]: saved file:  A_RES_67_291
[INFO]: saved file:  A_RES_48_186
[INFO]: saved file:  A_RES_59_231
[INFO]: saved file:  A_RES_57_262
[INFO]: saved file:  A_RES_58_4
[INFO]: saved file:  A_RES_58_211
[INFO]: saved file:  A_RES_60_42
[INFO]: saved file:  A_RES_48_96
[INFO]: saved file:  A_RES_72_252
[INFO]: saved file:  A_RES_51_241
[INFO]: saved file:  A_RES_56_259
[INFO]: saved file:  A_RES_52_194
[INFO]: saved file:  A_RES_54_249
[INFO]: saved file:  A_RES_48_163
[INFO]: saved file:  A_RES_53_190
[INFO]: saved file:  A_RES_58_135
[INFO]: saved file:  A_RES_48_172
[INFO]: saved file:  A_RES_60_74
[INFO]: saved file:  A_RES_55_151
[INFO]: saved file:  A_RES_58_179

>>> 1


[INFO]: saved file:  A
[INFO]: saved file:  A_RES_57_83
[INFO]: saved file:  A_RES_53_30
[INFO]: saved file:  A_RES_69_247
[INFO]: saved file:  A_RES_55_61
[INFO]: saved file:  A_RES_60_203
[INFO]: saved file:  A_RES_73_261
[INFO]: saved file:  A_RES_74_5
[INFO]: saved file:  A_RES_49_30
[INFO]: saved file:  A_RES_51_92
[INFO]: saved file:  A_RES_59_145
[INFO]: saved file:  A_RES_58_246
[INFO]: saved file:  A_RES_52_70
[INFO]: saved file:  A_RES_53_88
[INFO]: saved file:  A_RES_72_251
[INFO]: saved file:  A_RES_52_77
[INFO]: saved file:  A_RES_56_56
[INFO]: saved file:  A_RES_52_212
[INFO]: saved file:  A_RES_48_28
[INFO]: saved file:  A_RES_53_204
[INFO]: saved file:  A_RES_52_210
[INFO]: saved file:  A_RES_53_138
[INFO]: saved file:  A_RES_55_252
[INFO]: saved file:  A_RES_55_29
[INFO]: saved file:  A_RES_74_298
[INFO]: saved file:  A_RES_55_28
[INFO]: saved file:  A_RES_58_137
[INFO]: saved file:  A_RES_67_39
[INFO]: saved file:  A_RES_70_250
[INFO]: saved file:  A_RES_57_310
[INFO]

In [33]:
!open .

# Extracting clauses

For the cleanest clause extractions, some data sacrifices are to be made. The PDF documents, on each page, contain both headers and footers, which, when stitching the documents together, produces noise in the data. To increase the quality, we get rid of the last and first elements in the list of parsed clauses.

In [26]:
with open('files/text/A_RES_48_14.txt') as file:
    text = file.read()

In [27]:
# text cleaning
text = re.sub(' +', ' ', re.sub('\t|\n|\r', ' ', text)).strip()  # cleaning multiple spaces and indentation chars
text = re.sub('(\s+)', ' ', re.sub('[^a-z\s]+', ' ', text, flags=re.IGNORECASE)).strip()  # removing everything not in the english alphabet

## - with known clause-starters:

In [28]:
# clauses starters, taken from https://www.un.org/en/model-united-nations/drafting-resolutions
PREAMBS = [
    'Reiterating', 'Recognizing', 'Highlighting', 'Acknowledging', 'Affirming', 'Appreciating', 'Approving', 'Aware', 'Bearing in mind', 'Believing', 'Commending', 'Concerned', 'Conscious', 'Considering', 
    'Convinced', 'Desiring', 'Emphasizing', 'Expecting', 'Expressing', 'Fully aware', 'Guided by', 'Having adopted', 'Having considered', 'Having noted', 'Having reviewed', 'Mindful', 'Noting',
    'Noting with approval', 'Noting with concern', 'Noting with satisfaction', 'Observing', 'Realising', 'Recalling', 'Recognising', 'Seeking', 'Taking into consideration', 'Underlining', 'Welcoming', 'Whereas'
]
OPERS = [
    'Also requests', 'Further requests', 'Welcomes', 'Accepts', 'Adopts', 'Agrees', 'Appeals', 'Approves', 'Authorizes', 'Calls upon', 'Commends', 'Condemns', 'Considers', 'Decides', 'Declares', 'Determines',
    'Directs', 'Emphasizes', 'Encourages', 'Endorses', 'Expresses appreciation', 'Expresses hope', 'Invites', 'Notes', 'Notes with approval', 'Notes with concern', 'Notes with satisfaction', 'Proclaims',
    'Reaffirms', 'Recommends', 'Reminds', 'Repeals', 'Requests', 'Resolves', 'Suggests', 'Supports', 'Takes note', 'Urges'
]

In [31]:
clauses = re.split(f"({'|'.join(PREAMBS + OPERS)})", text) # split by preambular and operative clauses, keeping delimeter
clauses = clauses[1:]  # the first element is before the resolution starts
clauses = [a + ' ' + b for a, b in zip(clauses[::2], clauses[1::2])]
clauses = [re.sub(' +', ' ', x) for x in clauses]  # once more cleaning multiple spaces and indentation chars

In [32]:
for c in clauses:
    print('-' * 10)
    print(c)

len(clauses)

----------
Noting the statement of the Director General of the International Atomic Energy Agency of November in which he provided additional information on the main developments in the activities of the Agency during 
----------
Recognizing the importance of the work of the Agency in promoting the further application of nuclear energy for peaceful purposes as envisaged in its statute Also recognizing the special needs of the developing countries for technical assistance from the Agency in order to benefit effectively from the application of nuclear technology for peaceful purposes as well as from the contribution of nuclear energy to their economic development International Atomic Energy Agency The Annual Report for Austria July GC XXXVII transmitted to the members of the General Assembly by a note of the Secretary General A See Official Records of the General Assembly Forty eighth Session Plenary Meetings th meeting A PV A RES Page 
----------
Conscious of the importance of the work 

12

## - with part-of-speech-tagging:

with nltk, the relevant POS are 'VBG' (preamb) and 'VBZ' (operative)
result: no bueno...

In [151]:
nltk.download(['averaged_perceptron_tagger', 'punkt', 'tagsets'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/komron/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/komron/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to /Users/komron/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [149]:
words = nltk.word_tokenize(text)
tags = nltk.pos_tag(words)

clauses = list()
current_clause = list()

for w, t in tags:
    if (t == 'VBG' or t == 'VBZ') and w[0].isupper():
        clauses.append(current_clause)
        current_clause = list()  # reset after every clause-starter
    
    current_clause.append([w, t])

flat_clauses = list()

for i in clauses:
    c = []
    for j in i:
        c.append(j[0])
    flat_clauses.append(' '.join(c))

flat_clauses = flat_clauses[1:]

for c in flat_clauses:
    print("-" * 10)
    print(c)

tags

----------
Noting the statement of the Director General of the International Atomic Energy Agency of 1 November 1993 , 2/ in which he provided additional information on the main developments in the activities of the Agency during 1993 ,
----------
Recognizing the importance of the work of the Agency in promoting the further application of nuclear energy for peaceful purposes , as envisaged in its statute , Also recognizing the special needs of the developing countries for technical assistance from the Agency in order to benefit effectively from the application of nuclear technology for peaceful purposes as well as from the contribution of nuclear energy to their economic development , ____________ 1/ International Atomic Energy Agency , The Annual Report for 1992 ( Austria , July 1993 ) ( GC ( XXXVII ) /1060 ) ; transmitted to the members of the General Assembly by a note of the Secretary-General ( A/48/341 ) . 2/ See Official Records of the General Assembly , Forty-eighth Session , Pl

[('AA', 'NNP'),
 ('UUNNIITTEEDD', 'NNP'),
 ('NNAATTIIOONNSS', 'NNP'),
 ('General', 'NNP'),
 ('Assembly', 'NNP'),
 ('Distr', 'NNP'),
 ('.', '.'),
 ('GENERAL', 'NNP'),
 ('A/RES/48/14', 'NNP'),
 ('11', 'CD'),
 ('November', 'NNP'),
 ('1993', 'CD'),
 ('Forty-eighth', 'NNP'),
 ('session', 'NN'),
 ('Agenda', 'NNP'),
 ('item', 'NN'),
 ('14', 'CD'),
 ('RESOLUTION', 'NNP'),
 ('ADOPTED', 'NNP'),
 ('BY', 'NNP'),
 ('THE', 'NNP'),
 ('GENERAL', 'NNP'),
 ('ASSEMBLY', 'NNP'),
 ('[', 'VBD'),
 ('without', 'IN'),
 ('reference', 'NN'),
 ('to', 'TO'),
 ('a', 'DT'),
 ('Main', 'NNP'),
 ('Committee', 'NNP'),
 ('(', '('),
 ('A/48/L.13', 'NNP'),
 ('and', 'CC'),
 ('Corr.1', 'NNP'),
 ('and', 'CC'),
 ('Add.1', 'NNP'),
 (')', ')'),
 (']', 'VBD'),
 ('48/14', 'CD'),
 ('.', '.'),
 ('Report', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('International', 'NNP'),
 ('Atomic', 'NNP'),
 ('Energy', 'NNP'),
 ('Agency', 'NNP'),
 ('The', 'DT'),
 ('General', 'NNP'),
 ('Assembly', 'NNP'),
 (',', ','),
 ('Having', 'NNP'),
 ('received', 

In [152]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or